<center><h1><b>5주차 ✅ 폐기물 환경오염 데이터 분석 : 생활&사업장 폐기물</b></h1></center>

---
----

<center><h2><b>데이터 초기 전처리</b></h2></center>


### Tidy Data

In [2]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# import cufflinks as cf


In [3]:
# 데이터 path 확인
from glob import glob

file_list = glob('data/waste/*.csv')
file_list

['data/waste/business_dis_19.csv',
 'data/waste/business_hld_20.csv',
 'data/waste/business_hld_19.csv',
 'data/waste/business_dis_20.csv',
 'data/waste/household_20.csv',
 'data/waste/household_19.csv']

In [26]:
# 각 데이터프레임 종류
names = [
    '사업장배출시설',
    '사업장생활',
    '사업장생활',
    '사업장배출시설',
    '생활',
    '생활'
]

In [5]:
# 폴더 안 데이터 모두 가져오기
dfs = [pd.read_csv(path, encoding='utf-8') for path in file_list]
dfs[0].head()

,시도별(1),항목별(2),2019
0,강원,기타,217.2
1,강원,매립,628.8
2,강원,소각,160.8
3,강원,소계,12341.6
4,강원,재활용,11334.8


In [32]:
# tidy data + 컬럼명 변경 함수
def do_tidy_data(df, name):
    cols = ['시도명', '배출방식', '시점', name+'폐기물수']
    df_ = df.melt(id_vars=[df.columns[0], df.columns[1]])
    df_.columns = cols
    return df_

do_tidy_data(dfs[0], '사업장배출시설')

,시도명,배출방식,시점,사업장배출시설폐기물수
0,강원,기타,2019,217.2
1,강원,매립,2019,628.8
2,강원,소각,2019,160.8
3,강원,소계,2019,12341.6
4,강원,재활용,2019,11334.8
...,...,...,...,...
85,충북,기타,2019,1248.2
86,충북,매립,2019,908.0
87,충북,소각,2019,2201.4
88,충북,소계,2019,16573.2


In [38]:
# tidy 데이터 만들기
dfs_tidy = [do_tidy_data(df, names[i]) for i, df in enumerate(dfs)]
dfs_tidy[5].head()

,시도명,배출방식,시점,생활폐기물수
0,강원,계,2019,1866.3
1,강원,기타,2019,16.4
2,강원,매립,2019,1127.6
3,강원,소각,2019,836.0
4,강원,소계,2019,1866.3


In [46]:
# 소계 삭제하기
for i, df in enumerate(dfs_tidy):
    dfs_tidy[i] = df[~(df['배출방식'] == '소계')]    

In [62]:
# 2020년 폐기물
df_20  = pd.merge(pd.merge(dfs_tidy[1], dfs_tidy[3]), dfs_tidy[4])
df_20.head()

,시도명,배출방식,시점,사업장생활폐기물수,사업장배출시설폐기물수,생활폐기물수
0,강원도,기타,2020,50140.8,98363.4,6586.4
1,강원도,매립,2020,32602.6,245895.4,369244.6
2,강원도,소각,2020,20117.0,74140.4,322714.0
3,강원도,재활용,2020,212098.6,4083822.2,636822.0
4,경기도,기타,2020,550924.8,1767479.2,28812.2


In [63]:
# 2019년 폐기물
df_19  = pd.merge(pd.merge(dfs_tidy[0], dfs_tidy[2]), dfs_tidy[5])
df_19.head()

,시도명,배출방식,시점,사업장배출시설폐기물수,사업장생활폐기물수,생활폐기물수
0,강원,기타,2019,217.2,84.2,16.4
1,강원,매립,2019,628.8,115.4,1127.6
2,강원,소각,2019,160.8,62.0,836.0
3,강원,재활용,2019,11334.8,507.6,1752.6
4,경기,기타,2019,3278.4,496.8,74.6


In [66]:
# 데이터 저장
df_19.to_csv('output/waste/waste_2019.csv', index=False)
df_20.to_csv('output/waste/waste_2020.csv', index=False)

----

### 시도 + 광역시

In [3]:
# 19년도 데이터 광역시 명 다름
df_19 = pd.read_csv('output/waste/waste_2019.csv')
df_19.head()

,시도명,배출방식,시점,사업장배출시설폐기물수,사업장생활폐기물수,생활폐기물수
0,강원,기타,2019,217.2,84.2,16.4
1,강원,매립,2019,628.8,115.4,1127.6
2,강원,소각,2019,160.8,62.0,836.0
3,강원,재활용,2019,11334.8,507.6,1752.6
4,경기,기타,2019,3278.4,496.8,74.6


In [5]:
df_19['시도명'].unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [9]:
sidos = {
'강원' : '강원도',
'경기': '경기도',
'경남': '경상남도',
'경북': '경상북도',
'광주': '광주광역시',
'대구': '대구광역시',
'대전': '대전광역시',
'부산': '부산광역시',
'서울': '서울특별시',
'세종': '세종특별자치시',
'울산': '울산광역시',
'인천': '인천광역시',
'전남': '전라남도',
'전북': '전라북도',
'제주': '제주특별차치도',
'충남': '충청남도',
'충북': '충청북도'
}

df_19['시도명'] = df_19['시도명'].map(sidos)
df_19.head()

,시도명,배출방식,시점,사업장배출시설폐기물수,사업장생활폐기물수,생활폐기물수
0,강원도,기타,2019,217.2,84.2,16.4
1,강원도,매립,2019,628.8,115.4,1127.6
2,강원도,소각,2019,160.8,62.0,836.0
3,강원도,재활용,2019,11334.8,507.6,1752.6
4,경기도,기타,2019,3278.4,496.8,74.6


In [10]:
df_19.to_csv('output/waste/waste_2019.csv', index=False)

----